# Inflation Derivatives

This notebook demonstrates the inflation derivatives stack in PyQuantLib:

1. Bootstrap a zero-coupon inflation curve from US CPI swap market data
2. Build a year-on-year (YoY) inflation curve
3. Price zero-coupon and year-on-year inflation swaps
4. Price YoY inflation caps, floors, and collars with the Black engine
5. Apply seasonal adjustments to the inflation curve
6. Compute inflation DV01 via pillar-by-pillar curve bumping

In [1]:
import pyquantlib as ql
import pandas as pd
import numpy as np
from datetime import date

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

PyQuantLib 0.4.0.dev0 (QuantLib 1.40)


## Setup

Inflation indices are published with a lag. US CPI is released monthly with
a roughly 3-month observation lag: a swap starting in January 2025 references
CPI from October 2024.

In [2]:
calendar = ql.TARGET()
today = date(2025, 1, 15)
ql.Settings.evaluationDate = today

dc = ql.Actual365Fixed()
obs_lag = ql.Period(3, ql.Months)

# Flat nominal discount curve at 4%
nominal_rate = 0.04
nominal_curve = ql.FlatForward(today, nominal_rate, dc)
nominal_handle = ql.YieldTermStructureHandle(nominal_curve)

print(f"Evaluation date:  {today}")
print(f"Observation lag:  {obs_lag}")
print(f"Nominal rate:     {nominal_rate:.2%}")

Evaluation date:  2025-01-15
Observation lag:  3M
Nominal rate:     4.00%


## US CPI Index and Historical Fixings

Create a `USCPI` index and add the base fixing that anchors the inflation curve.
Real market data would include a full history of monthly CPI releases.

In [3]:
cpi_index = ql.USCPI()

# Base CPI level (October 2024 release)
base_cpi = 315.0
base_date = ql.Date(1, ql.October, 2024)
cpi_index.addFixing(base_date, base_cpi)

print(f"Index:     {cpi_index.name()}")
print(f"Base date: {base_date}")
print(f"Base CPI:  {base_cpi}")

Index:     USA CPI
Base date: October 1st, 2024
Base CPI:  315.0


## Bootstrapping the Zero-Coupon Inflation Curve

Zero-coupon inflation swap (ZCIS) rates are the primary market instrument for
building an inflation curve. Each helper represents a market-quoted ZCIS rate
at a given maturity.

A ZCIS exchanges a fixed compounded payment against the realized CPI return
over the swap tenor. The fixed rate is the "breakeven inflation" rate.

In [4]:
# Market-quoted zero-coupon inflation swap rates
zcis_market = [
    (1,  0.0220),
    (2,  0.0235),
    (3,  0.0248),
    (5,  0.0260),
    (7,  0.0268),
    (10, 0.0275),
    (15, 0.0280),
    (20, 0.0282),
    (30, 0.0285),
]

helpers = []
for years, rate in zcis_market:
    maturity = ql.Date(15, ql.January, 2025) + ql.Period(years, ql.Years)
    helpers.append(ql.ZeroCouponInflationSwapHelper(
        rate, obs_lag, maturity, calendar,
        ql.ModifiedFollowing, dc, cpi_index, ql.CPI.Flat,
    ))

zero_curve = ql.PiecewiseZeroInflationCurve(
    ql.Date(15, ql.January, 2025), base_date, ql.Monthly, dc, helpers,
)
zero_curve.enableExtrapolation()

print(f"Zero-coupon inflation curve bootstrapped from {len(helpers)} ZCIS helpers")

Zero-coupon inflation curve bootstrapped from 9 ZCIS helpers


### Inspecting the Bootstrapped Curve

Extract the bootstrapped nodes and query zero-coupon inflation rates at
various maturities.

In [5]:
# Bootstrapped curve nodes
nodes = zero_curve.nodes()
node_df = pd.DataFrame(nodes, columns=["Date", "ZC Rate"])
node_df["ZC Rate"] = node_df["ZC Rate"].map("{:.4%}".format)
print("Bootstrapped nodes:")
print(node_df.to_string(index=False))

Bootstrapped nodes:
             Date ZC Rate
October 1st, 2024 2.2000%
October 1st, 2025 2.2000%
October 1st, 2026 2.3500%
October 1st, 2027 2.4800%
October 1st, 2029 2.6000%
October 1st, 2031 2.6800%
October 1st, 2034 2.7500%
October 1st, 2039 2.8000%
October 1st, 2044 2.8200%
October 1st, 2054 2.8500%


In [6]:
# Query zero-coupon inflation rates at semi-annual intervals
query_years = np.arange(1, 31, 0.5)
zc_rates = []
for y in query_years:
    target = ql.Date(15, ql.January, 2025) + ql.Period(int(y * 12), ql.Months)
    zc_rates.append(zero_curve.zeroRate(target))

rate_df = pd.DataFrame({"Tenor (years)": query_years, "ZC Inflation Rate": zc_rates})

print("Inflation term structure (selected tenors):")
print(rate_df.iloc[::4].to_string(
    index=False,
    formatters={
        "Tenor (years)": "{:.1f}".format,
        "ZC Inflation Rate": "{:.4%}".format,
    },
))

Inflation term structure (selected tenors):
Tenor (years) ZC Inflation Rate
          1.0           2.2378%
          3.0           2.4951%
          5.0           2.6101%
          7.0           2.6859%
          9.0           2.7326%
         11.0           2.7625%
         13.0           2.7825%
         15.0           2.8010%
         17.0           2.8090%
         19.0           2.8170%
         21.0           2.8238%
         23.0           2.8298%
         25.0           2.8358%
         27.0           2.8418%
         29.0           2.8478%


## Year-on-Year Inflation Curve

The YoY curve is built from year-on-year inflation swap helpers. A YoY swap
exchanges annual fixed payments against realized YoY CPI returns.

The `YoYInflationIndex` wraps a zero-coupon index to compute the ratio-based
year-on-year rate: $r_t = I(t) / I(t-1) - 1$.

In [7]:
yoy_index = ql.YoYInflationIndex(ql.USCPI())

# Market-quoted YoY inflation swap rates
yoy_market = [
    (1,  0.0215),
    (2,  0.0230),
    (3,  0.0242),
    (5,  0.0255),
    (7,  0.0262),
    (10, 0.0270),
]

yoy_helpers = []
for years, rate in yoy_market:
    maturity = ql.Date(15, ql.January, 2025) + ql.Period(years, ql.Years)
    yoy_helpers.append(ql.YearOnYearInflationSwapHelper(
        rate, obs_lag, maturity, calendar,
        ql.ModifiedFollowing, dc, yoy_index,
        ql.CPI.Flat, nominal_handle,
    ))

yoy_curve = ql.PiecewiseYoYInflationCurve(
    ql.Date(15, ql.January, 2025), base_date, 0.022, ql.Monthly, dc, yoy_helpers,
)
yoy_curve.enableExtrapolation()

# Verify calibration by repricing YoY swaps at each market tenor
yoy_linked = ql.YoYInflationIndex(
    ql.USCPI(),
    ql.YoYInflationTermStructureHandle(yoy_curve),
)

fair_rates = []
for years, mkt_rate in yoy_market:
    start = ql.Date(15, ql.January, 2025)
    end = start + ql.Period(years, ql.Years)
    sched = ql.MakeSchedule(
        effectiveDate=start, terminationDate=end,
        tenor=ql.Period(1, ql.Years), calendar=calendar,
    )
    swap = ql.YearOnYearInflationSwap(
        ql.SwapType.Payer, 1_000_000.0,
        sched, mkt_rate, dc,
        sched, yoy_linked, obs_lag, ql.CPI.Flat,
        0.0, dc, calendar,
    )
    swap.setPricingEngine(ql.DiscountingSwapEngine(nominal_handle))
    fair_rates.append(swap.fairRate())

yoy_df = pd.DataFrame({
    "Tenor": [f"{y}Y" for y, _ in yoy_market],
    "Market Rate": [r for _, r in yoy_market],
    "Fair Rate": fair_rates,
})
print("YoY inflation curve calibration (swap repricing):")
print(yoy_df.to_string(
    index=False,
    formatters={"Market Rate": "{:.4%}".format, "Fair Rate": "{:.4%}".format},
))

YoY inflation curve calibration (swap repricing):
Tenor Market Rate Fair Rate
   1Y     2.1500%   2.1500%
   2Y     2.3000%   2.3000%
   3Y     2.4200%   2.4204%
   5Y     2.5500%   2.5499%
   7Y     2.6200%   2.6200%
  10Y     2.7000%   2.6999%


## Pricing a Zero-Coupon Inflation Swap

A 5-year ZCIS on \$10M notional. The **payer** pays fixed and receives the
CPI-linked return. At maturity, the inflation leg pays:

$$N \times \left(\frac{\text{CPI}(T)}{\text{CPI}(0)} - 1\right)$$

while the fixed leg pays $N \times \left((1 + r)^T - 1\right)$.

Using the bootstrapped zero-coupon curve, we link the CPI index to the curve
and price the swap with a standard discounting engine.

In [8]:
# Link CPI index to the bootstrapped curve
cpi_for_swap = ql.USCPI(ql.ZeroInflationTermStructureHandle(zero_curve))
cpi_for_swap.addFixing(base_date, base_cpi)

nominal = 10_000_000.0
fixed_rate = 0.0260  # pay 2.60% fixed
maturity = ql.Date(15, ql.January, 2030)

zcis = ql.ZeroCouponInflationSwap(
    ql.SwapType.Payer, nominal,
    ql.Date(15, ql.January, 2025), maturity,
    calendar, ql.ModifiedFollowing, dc,
    fixed_rate, cpi_for_swap, obs_lag, ql.CPI.Flat,
)

swap_engine = ql.DiscountingSwapEngine(nominal_handle)
zcis.setPricingEngine(swap_engine)

print(f"5Y Zero-Coupon Inflation Swap")
print(f"  Notional:        ${nominal:,.0f}")
print(f"  Fixed rate:      {fixed_rate:.2%}")
print(f"  NPV:             ${zcis.NPV():,.2f}")
print(f"  Fair rate:       {zcis.fairRate():.4%}")
print(f"  Fixed leg NPV:   ${zcis.fixedLegNPV():,.2f}")
print(f"  Inflation leg NPV: ${zcis.inflationLegNPV():,.2f}")

5Y Zero-Coupon Inflation Swap
  Notional:        $10,000,000
  Fixed rate:      2.60%
  NPV:             $0.00
  Fair rate:       2.6000%
  Fixed leg NPV:   $1,121,685.67
  Inflation leg NPV: $-1,121,685.67


## Pricing a Year-on-Year Inflation Swap

A 5-year YoY inflation swap on \$10M notional. Each year, the inflation leg
pays the realized year-on-year CPI change, while the fixed leg pays a constant rate.

In [9]:
# Link YoY index to the bootstrapped curve
yoy_for_swap = ql.YoYInflationIndex(
    ql.USCPI(),
    ql.YoYInflationTermStructureHandle(yoy_curve),
)

yoy_start = ql.Date(15, ql.January, 2025)
yoy_maturity = ql.Date(15, ql.January, 2030)
yoy_fixed_rate = 0.0255

annual_schedule = ql.MakeSchedule(
    effectiveDate=yoy_start, terminationDate=yoy_maturity,
    tenor=ql.Period(1, ql.Years), calendar=calendar,
)

yoy_swap = ql.YearOnYearInflationSwap(
    ql.SwapType.Payer, nominal,
    annual_schedule, yoy_fixed_rate, dc,
    annual_schedule, yoy_for_swap, obs_lag, ql.CPI.Flat,
    0.0, dc, calendar,
)
yoy_swap.setPricingEngine(swap_engine)

print(f"5Y Year-on-Year Inflation Swap")
print(f"  Notional:     ${nominal:,.0f}")
print(f"  Fixed rate:   {yoy_fixed_rate:.2%}")
print(f"  NPV:          ${yoy_swap.NPV():,.2f}")
print(f"  Fair rate:    {yoy_swap.fairRate():.4%}")
print(f"  Fair spread:  {yoy_swap.fairSpread():.4%}")

5Y Year-on-Year Inflation Swap
  Notional:     $10,000,000
  Fixed rate:   2.55%
  NPV:          $-25.97
  Fair rate:    2.5499%
  Fair spread:  0.0001%


## Pricing a YoY Inflation Cap

A YoY inflation cap protects against annual inflation exceeding a strike level.
Each caplet pays $N \times \max(\text{YoY}_t - K, 0)$ at the end of each period.

Pricing uses the `YoYInflationBlackCapFloorEngine` with a constant YoY
optionlet volatility surface.

In [10]:
# YoY optionlet volatility surface (constant 10% lognormal vol)
yoy_vol = 0.10
vol_surface = ql.ConstantYoYOptionletVolatility(
    yoy_vol, 2, calendar, ql.ModifiedFollowing, dc,
    obs_lag, ql.Monthly, False,
)
vol_handle = ql.YoYOptionletVolatilitySurfaceHandle(vol_surface)

# Build the YoY inflation leg
yoy_leg = ql.yoyInflationLeg(
    annual_schedule, calendar, yoy_for_swap, obs_lag, ql.CPI.Flat,
)
yoy_leg = yoy_leg.withNotionals([nominal]).withPaymentDayCounter(dc).build()

# Attach a coupon pricer (requires both vol handle and nominal curve handle)
pricer = ql.BlackYoYInflationCouponPricer(
    capletVol=vol_handle, nominalTermStructure=nominal_handle,
)
ql.setCouponPricer(yoy_leg, pricer)

# Create the cap
cap_strike = 0.03
yoy_cap = ql.YoYInflationCap(yoy_leg, [cap_strike])

cap_engine = ql.YoYInflationBlackCapFloorEngine(
    yoy_for_swap, vol_handle, nominal_handle,
)
yoy_cap.setPricingEngine(cap_engine)

print(f"5Y YoY Inflation Cap")
print(f"  Notional:   ${nominal:,.0f}")
print(f"  Strike:     {cap_strike:.2%}")
print(f"  Vol:        {yoy_vol:.0%} (lognormal)")
print(f"  NPV:        ${yoy_cap.NPV():,.2f}")

5Y YoY Inflation Cap
  Notional:   $10,000,000
  Strike:     3.00%
  Vol:        10% (lognormal)
  NPV:        $31,928.69


### Cap vs Floor vs Collar

Compare a cap, floor, and collar at the same strikes.

In [11]:
floor_strike = 0.01

yoy_floor = ql.YoYInflationFloor(yoy_leg, [floor_strike])
yoy_floor.setPricingEngine(cap_engine)

yoy_collar = ql.YoYInflationCollar(yoy_leg, [cap_strike], [floor_strike])
yoy_collar.setPricingEngine(cap_engine)

results = pd.DataFrame({
    "Instrument": ["Cap (3%)", "Floor (1%)", "Collar (1%-3%)"],
    "NPV": [yoy_cap.NPV(), yoy_floor.NPV(), yoy_collar.NPV()],
})
results["NPV"] = results["NPV"].map("${:,.2f}".format)
print(results.to_string(index=False))

    Instrument        NPV
      Cap (3%) $31,928.69
    Floor (1%)      $0.01
Collar (1%-3%) $31,928.68


## Seasonality

CPI exhibits seasonal patterns (e.g., energy and food costs vary by month).
PyQuantLib supports `MultiplicativePriceSeasonality` to adjust the inflation
curve for these effects.

In [12]:
# Stylized monthly seasonal factors (sum to ~12.0)
seasonal_factors = [
    1.003,  # Jan - winter energy
    1.005,  # Feb
    1.008,  # Mar - spring
    1.005,  # Apr
    1.002,  # May
    0.998,  # Jun
    0.995,  # Jul - summer lull
    0.993,  # Aug
    0.996,  # Sep
    0.999,  # Oct - back to school
    0.998,  # Nov
    0.998,  # Dec
]

seasonality = ql.MultiplicativePriceSeasonality(
    ql.Date(1, ql.January, 2024), ql.Monthly, seasonal_factors,
)

# Build a curve with seasonality
dates = [base_date] + [
    ql.Date(15, ql.January, 2025) + ql.Period(y, ql.Years)
    for y in [1, 2, 3, 5, 7, 10]
]
rates = [0.022, 0.022, 0.0235, 0.0248, 0.0260, 0.0268, 0.0275]

seasonal_curve = ql.ZeroInflationCurve(
    ql.Date(15, ql.January, 2025), dates, rates, ql.Monthly, dc, seasonality,
)

plain_curve = ql.ZeroInflationCurve(
    ql.Date(15, ql.January, 2025), dates, rates, ql.Monthly, dc,
)

# Compare zero rates
comparison = []
for y in [1, 2, 3, 5, 7, 10]:
    target = ql.Date(15, ql.January, 2025) + ql.Period(y, ql.Years)
    comparison.append({
        "Tenor": f"{y}Y",
        "Plain": plain_curve.zeroRate(target),
        "Seasonal": seasonal_curve.zeroRate(target),
    })

comp_df = pd.DataFrame(comparison)
comp_df["Diff (bp)"] = (comp_df["Seasonal"] - comp_df["Plain"]) * 10000
print("Impact of seasonality on zero-coupon inflation rates:")
print(comp_df.to_string(
    index=False,
    formatters={
        "Plain": "{:.4%}".format,
        "Seasonal": "{:.4%}".format,
        "Diff (bp)": "{:+.2f}".format,
    },
))

Impact of seasonality on zero-coupon inflation rates:
Tenor   Plain Seasonal Diff (bp)
   1Y 2.2000%  2.5267%    +32.67
   2Y 2.3442%  2.5260%    +18.18
   3Y 2.4750%  2.6010%    +12.60
   5Y 2.5977%  2.6758%     +7.81
   7Y 2.6785%  2.7350%     +5.66
  10Y 2.7491%  2.7891%     +4.00


## Sensitivity: Bumping the Inflation Curve

Compute the inflation DV01 of the zero-coupon inflation swap by bumping each
ZCIS helper rate up by 1 bp and repricing.

In [13]:
base_npv = zcis.NPV()
bump = 0.0001  # 1 bp

sensitivities = []
for i, (years, rate) in enumerate(zcis_market):
    # Rebuild helpers with one rate bumped
    bumped_cpi = ql.USCPI()
    bumped_cpi.addFixing(base_date, base_cpi)

    bumped_helpers = []
    for j, (y, r) in enumerate(zcis_market):
        bumped_rate = r + bump if j == i else r
        mat = ql.Date(15, ql.January, 2025) + ql.Period(y, ql.Years)
        bumped_helpers.append(ql.ZeroCouponInflationSwapHelper(
            bumped_rate, obs_lag, mat, calendar,
            ql.ModifiedFollowing, dc, bumped_cpi, ql.CPI.Flat,
        ))

    bumped_curve = ql.PiecewiseZeroInflationCurve(
        ql.Date(15, ql.January, 2025), base_date, ql.Monthly, dc, bumped_helpers,
    )

    # Reprice the swap on the bumped curve
    bumped_cpi_linked = ql.USCPI(ql.ZeroInflationTermStructureHandle(bumped_curve))
    bumped_cpi_linked.addFixing(base_date, base_cpi)

    bumped_zcis = ql.ZeroCouponInflationSwap(
        ql.SwapType.Payer, nominal,
        ql.Date(15, ql.January, 2025), maturity,
        calendar, ql.ModifiedFollowing, dc,
        fixed_rate, bumped_cpi_linked, obs_lag, ql.CPI.Flat,
    )
    bumped_zcis.setPricingEngine(swap_engine)

    dv01 = bumped_zcis.NPV() - base_npv
    sensitivities.append({"Pillar": f"{years}Y", "DV01": dv01})

sens_df = pd.DataFrame(sensitivities)
print("Inflation DV01 (NPV change per 1bp bump in each ZCIS rate):")
print(sens_df.to_string(index=False, formatters={"DV01": "${:,.2f}".format}))

Inflation DV01 (NPV change per 1bp bump in each ZCIS rate):
Pillar       DV01
    1Y      $0.00
    2Y      $0.00
    3Y      $0.00
    5Y $-4,539.48
    7Y      $0.00
   10Y      $0.00
   15Y      $0.00
   20Y      $0.00
   30Y      $0.00


## Summary

This notebook demonstrated the full inflation derivatives workflow:

| Step | PyQuantLib Classes |
|------|--------------------|
| CPI index + fixings | `USCPI`, `addFixing` |
| Zero-coupon curve bootstrap | `ZeroCouponInflationSwapHelper`, `PiecewiseZeroInflationCurve` |
| YoY curve bootstrap | `YearOnYearInflationSwapHelper`, `PiecewiseYoYInflationCurve` |
| Zero-coupon inflation swap | `ZeroCouponInflationSwap` |
| YoY inflation swap | `YearOnYearInflationSwap` |
| YoY cap/floor/collar | `YoYInflationCap`, `YoYInflationFloor`, `YoYInflationCollar` |
| Black cap/floor engine | `YoYInflationBlackCapFloorEngine`, `ConstantYoYOptionletVolatility` |
| Seasonality | `MultiplicativePriceSeasonality` |
| Sensitivity | Pillar-by-pillar DV01 via curve bumping |